## A simple swarm

In [1]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

In [2]:
from core.agent import *
from core.world import * 
from core.map import *
import numpy as np
from core.render import * 

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.9)


In [3]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(1000)]
    for agent in swarm:
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [4]:
world = World(dims = (100, 100),
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
world.reset()

In [5]:
# Test movements by doing random actions
def update():
    world.update()
    # Comms protocol (receiving)
    for agent in world.agents: 
        received_messages = agent.get_messages()
        for msg in received_messages: 
            agent.add_relation(msg.sender, msg.message)
        agent.clear_messages()

    for agent in world.agents:
        action_choice = np.random.randint(0, 4)
        choice = np.random.randint(1, 5)

        match (action_choice): 
            case 1: agent.move(choice)
            case 2: agent.pick_up(choice)
            case 3: agent.put_down(choice)
    
    # Comms protocol (sending )
    for agent in world.agents:
        neighbors = agent.agents_in_range 

        # Send agents
        if len (neighbors) > 0:
            choice = np.random.choice(neighbors)
            tgt_agent = world.get_agent(choice)
            agent.talk_to(tgt_agent)

update()

In [6]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [7]:
swarm = world.agents

In [8]:
# Validate the observation space provided is sensible
obs = swarm[123].local_observation

print(obs.nearby_agents, obs.nearby_agents.shape)
print(obs.resource_types, obs.resource_types.shape)

[[  0 833   0 620   0   0   0]
 [  0   0   0 774 772   0   0]
 [  0 238   0   0 241   0  78]
 [  0   0   0 124   0   0   0]
 [  0   0   0   0 899   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0  97   0   0   0   0]] (7, 7)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] (7, 7)


In [9]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.current_position
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.agents))

1000 1000


In [10]:
# Check for resources
rsrc_qty = world._resource_map._resource_quantity_map
qtys = []

for i in range(rsrc_qty.shape[0]):
    for j in range(rsrc_qty.shape[1]):
        if rsrc_qty[i, j] > 0:
            qtys.append(rsrc_qty[i, j])

print(qtys)

[3.011869943127354, 5.637992604757388, 7.273187735914448, 2.051947488302212, 5.3051494178152145, 3.6111219741336673, 1.0, 5.508311191751396, 3.2252064628223494, 4.957105112537895, 7.36729998014388, 5.994836592161742, 2.511582617259695, 4.827449852793629, 5.336759058430063, 8.507111174215911, 2.2829728707937496, 5.706888398772774, 2.5198879598569235, 5.70162526792144, 6.924994131589037, 7.104140434365787, 4.105745720430193, 8.358279202215993, 6.6069779291028015, 5.976700556339594, 8.908731212947375, 8.959081363627508, 3.2084800203598958, 5.572074505110636, 4.8633965977622875, 4.244395269701007, 4.664398940622175, 6.7461137368074695, 3.6203093753952773, 2.421097547255143, 7.533265981614248, 7.909395895833089, 7.506128340476906, 5.6286867787104935, 5.615087507565144, 2.5621062336908174, 4.841314742211373, 2.5394272545151004, 6.43191245029848, 1.5700281092500474, 1.2475536594127878, 3.0115752601890247, 1.0, 1.8766765267359613, 5.28874261214731, 4.19070375915258, 4.1697705017027715, 3.72972

In [11]:
# Check for agent state
state = swarm[0]._current_state.inventory
print(state)


[]


# Perftest

In [12]:
import cProfile
def stress_test():
    np.random.seed(42)
    world.reset()
    for _ in range(0, 1000):
        update()

In [13]:

cProfile.run('stress_test()', sort = 'time')

In [14]:
cProfile.run('world.update()', sort = 'time')

         26014 function calls in 0.008 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.002    0.000    0.002    0.000 world.py:142(_get_nearby_resources)
     1000    0.001    0.000    0.002    0.000 world.py:130(_get_nearby_agents)
        1    0.001    0.001    0.008    0.008 world.py:51(update)
        1    0.001    0.001    0.001    0.001 world.py:88(_update_movement)
     5000    0.000    0.000    0.000    0.000 {built-in method builtins.max}
     1000    0.000    0.000    0.001    0.000 models.py:11(forward)
        1    0.000    0.000    0.001    0.001 world.py:188(get_presence_mask)
     4000    0.000    0.000    0.000    0.000 {built-in method builtins.min}
     5000    0.000    0.000    0.000    0.000 agent.py:282(current_position_const)
     1000    0.000    0.000    0.000    0.000 agent.py:223(set_observation)
     1000    0.000    0.000    0.000    0.000 agent.py:217(reset_for_next_action)
     3